# KG-RAG: Knowledge Graph + Vector Store + LLM (Autonomous RAG)
This notebook shows how to:
- Extract entity triples from text using an LLM
- Store them in a knowledge graph (Neo4j or in-memory)
- Store document chunks in a vector database (FAISS)
- Query both sources using a user prompt
- Feed facts + documents to LLM for contextual answers

In [1]:
# Install dependencies (if running in Colab)
!pip install langchain openai faiss-cpu tiktoken neo4j networkx

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached charset_normalizer-3.4.1-cp313-cp313-win_amd64.whl.metadata (36 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 14.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/599.1 kB ? eta -:--:--
   --------------------------------------- 599.1/599.1 kB 15.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   --------------------- ------------------ 7.3/13.7 MB 35.0 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.7 MB 32.8 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 26.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.7 kB 

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
import os
from langchain_community.llms import Ollama

llm = Ollama(model="phi3")

## Step 1: Simulate Documents

In [ ]:
documents = [
    {
        "title": "Spring Fling 10K Overview",
        "url": "https://runclub.com/spring-fling",
        "content": "The Spring Fling 10K is led by Alan and takes place in Prospect Park."
    },
    {
        "title": "Alan's Profile",
        "url": "https://runclub.com/alan",
        "content": "Alan is a runner and site leader for Spring Fling 10K. See details here: https://runclub.com/spring-fling"
    }
]

## Step 2: Extract Triples from Documents Using LLM

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

triple_extraction_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
Extract factual triples from the following text.
Return as a list of JSON dicts: [{'subject': ..., 'predicate': ..., 'object': ...}]

Text: {text}
Triples:
"""
)

llm = ChatOpenAI(temperature=0)
chain = LLMChain(llm=llm, prompt=triple_extraction_prompt)

all_triples = []
for doc in documents:
    result = chain.run(text=doc['content'])
    print(f"Triples from {doc['title']}:\n{result}\n")
    all_triples.append(result)

## Step 3: (Optional) Store Triples in Graph Database (Neo4j or NetworkX)

In [ ]:
# Example for Neo4j (optional)
# graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="your-password")
# graph.query("Cypher query here")

## Step 4: Vector Store (FAISS)

In [ ]:
lc_documents = [
    Document(
        page_content=doc['content'],
        metadata={"title": doc['title'], "url": doc['url']}
    ) for doc in documents
]

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(lc_documents)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("faiss_kg_rag")

## Step 5: Final Query - Combine KG and Vector Context

In [ ]:
query = "Where does Alan lead events?"

# Simulated KG result
facts = [
    "Alan leads the Spring Fling 10K",
    "Spring Fling 10K takes place in Prospect Park"
]

# Vector DB search
results = vectorstore.similarity_search("Spring Fling 10K", k=2)

print("\nGraph Facts:")
for f in facts:
    print("-", f)

print("\nVector Search Results:")
for r in results:
    print(f"[{r.metadata['title']}]: {r.page_content}\n")